In [59]:
import re
from SolvingAdvent.HelferWichtel import HelferWichtel as Wichtel

TagFile = '07'
TestFile = '07Test'

Helfer = Wichtel()
PuzzleInput = Helfer.getPuzzleInput(TagFile)
TestInput = Helfer.getPuzzleInput(TestFile)
TestInputbyCommands = ''.join(TestInput).split('$')
TestInputbyCommands = [Input.strip().split('\n') for Input in TestInputbyCommands]
print(TestInputbyCommands)

PuzzleInputbyCommands = ''.join(PuzzleInput).split('$')
PuzzleInputbyCommands = [Input.strip().split('\n') for Input in PuzzleInputbyCommands]


[[''], ['cd /'], ['ls', 'dir a', '14848514 b.txt', '8504156 c.dat', 'dir d'], ['cd a'], ['ls', 'dir e', '29116 f', '2557 g', '62596 h.lst'], ['cd e'], ['ls', '584 i'], ['cd ..'], ['cd ..'], ['cd d'], ['ls', '4060174 j', '8033020 d.log', '5626152 d.ext', '7214296 k']]


In [60]:
class Nodes():
    types = {'dir','file'}
    
    def __init__(self, name:str = '', type:str='dir', parent:'Nodes' = None, size:int = 0, children:list['Nodes'] = None):
        self.name = name
        self.parent = parent
        self.type = type
        self.size = size
        self.children = children or []

    def addChild(self,NewNode:'Nodes'):
        self.children.append(NewNode)

    def getAllChildren(self) -> list['Nodes']:
        return self.children
    
    def getChildByName(self,name):
        Node = None
        for child in self.getAllChildren():
            if name == child.getName():
                Node = child
        return Node
    
    def getParent(self) -> 'Nodes':
        return self.parent

    def getName(self) -> str:
        return self.name

    def getSize(self) -> int:
        return int(self.size)

    def getType(self) -> str:
        return self.type

    def getAllChildrenNames(self) ->list[str]:
        childrenList = []
        for child in self.children:
            childrenList.append(child.getName())
        return childrenList


class TreeCrawler():
    def __init__(self, rootNode:'Nodes'= None):
        if not rootNode:
           rootNode = Nodes('root', 'dir')
           print('root node')
        self.activeNode = rootNode
        self.rootNode = rootNode
        pass

    def getActiveNode(self):
        return self.activeNode
    
    def setActiveNode(self,newNode:'Nodes'):
        self.activeNode = newNode

    def moveToParent(self):
        if self.getActiveNode().getName() == 'root':
            print('You are in root already')
            return
        else:
            self.setActiveNode(self.getActiveNode().getParent()) 

    def moveToChild(self,childNode:'Nodes'):
        if childNode.getType() != 'dir':
            raise Exception('Child is not Dir')
        else:
            self.setActiveNode(childNode)
            
    def moveToRoot(self):
        self.setActiveNode(self.rootNode)
    
    def moveToChildByString(self,childNode:str):
        for dir in self.getAllSubDirs():
            if dir.getName() == childNode:
                self.setActiveNode(dir)
        
    def getAllFiles(self) -> list['Nodes']:
        files= []
        for child in self.getActiveNode().getAllChildren():
            if child.getType() == 'file':
                files.append(child)
        return files
    
    def listAllFiles(self) -> list[str]:
        files = []
        for child in self.getActiveNode().getAllChildren():
            if child.getType() == 'file':
                files.append(str(child.getSize()) + ' ' + child.getName())
        return files

    def getAllSubDirs(self) -> list['Nodes']:
        dirs = []
        if type(self.getActiveNode()) is not Nodes:
            print(self.getActiveNode())
            raise Exception('Active Node ist kein Nodes Object')
        for child in self.getActiveNode().getAllChildren():
            if child.getType() == 'dir':
                dirs.append(child)
        return dirs

    def listAllSubDirs(self) -> list[str]:
        dirs = []
        for child in self.getActiveNode().getAllChildren():
            if child.getType() == 'dir':
                dirs.append(child.getType() + ' ' + child.getName())
        return dirs

    def createFile(self, fileName, fileSize):
        if any(file.getName() == fileName for file in self.getAllFiles()):
            print('Datei mit diesem Namen existiert bereits!')
            return
        else:
            newFile = Nodes(fileName,'file',self.getActiveNode(),fileSize)
            self.getActiveNode().addChild(newFile)
    
    def createDir(self, dirName):
        if any(dir.getName() == dirName for dir in self.getAllSubDirs()):
            print('Directory mit diesem Namen existiert bereits!')
            return
        else:
            newDir = Nodes(dirName,'dir',self.getActiveNode())
            self.getActiveNode().addChild(newDir)
    
    def riddleCommandParser(self, command:str, commandArg:str, consoleOutput:list[str]):
        if command == 'cd':
            if commandArg =='..':
                self.moveToParent()
                # print('Auf parent %s gewechset' %self.getActiveNode().getName())
                return
            else:
                if commandArg in [dir.getName() for dir in self.getAllSubDirs()]:
                    self.moveToChild(self.getActiveNode().getChildByName(commandArg))
                    # print('auf %s gewechselt' %self.getActiveNode().getName())
                    return
                if commandArg == '/':
                    self.setActiveNode(self.rootNode)
                    return
                else:
                    raise Exception('Dir not found')
        if command == 'ls':
            for output in consoleOutput:
                if output[0:3] == 'dir':
                    if output not in [dir.getName() for dir in self.getAllSubDirs()]:
                        self.createDir(output[4:])
                if any(re.findall("\d",output[0:3])):
                    splitOutput = output.split(' ')
                    size = int(splitOutput[0])
                    name = splitOutput[1]
                    if output not in [file.getName() for file in self.getAllFiles()]:
                        self.createFile(name,size)
        
    def commandOutputParser(self, batch):
        command = batch[0][0:2]
        commandArg = ''
        consoleOutput = []
        if command == 'cd':
            command, commandArg = batch[0].split(' ')
        consoleOutput = batch[1:]
        self.riddleCommandParser(command,commandArg,consoleOutput)
        
    def getSizeOfCurrentDirectory(self) -> int:
        searchTree = TreeCrawler(self.getActiveNode())
        sizeOfThisDirectory = 0
        allSubDirs = self.getAllSubDirs()
        allFiles = self.getAllFiles()
        for Dir in allSubDirs:
            searchTree.moveToChild(Dir)
            sizeOfThisDirectory += searchTree.getSizeOfCurrentDirectory()
            searchTree.moveToParent()
        for file in allFiles:
            sizeOfThisDirectory += file.getSize()
        return sizeOfThisDirectory
    
    def getSizeOfAllSubDirectories(self) -> list[int]:
        '''Returns List of sizes of all subdirectories + current directory'''
        searchTree = TreeCrawler(self.getActiveNode())
        sizes = []
        sizes.append(searchTree.getSizeOfCurrentDirectory())
        allSubDirs = self.getAllSubDirs()
        # print([dir.getName() for dir in allSubDirs])
        for Dir in allSubDirs:
            searchTree.moveToChild(Dir)
            sizes.extend(searchTree.getSizeOfAllSubDirectories())
            searchTree.moveToParent()
        return sizes
                

In [61]:
#Test Input

Tree = TreeCrawler()
for batch in TestInputbyCommands[:]:
    Tree.commandOutputParser(batch)
Tree.getSizeOfCurrentDirectory()
Tree.moveToRoot()
print('Suche beginnt in dieser Node: %s' %Tree.getActiveNode().getName())
sizeOfAllDirectories = Tree.getSizeOfAllSubDirectories()
print(sizeOfAllDirectories)
AnswerTest = [size for size in sizeOfAllDirectories if size <= 100000]
print(sum(AnswerTest))

root node
Suche beginnt in dieser Node: root
[48381165, 94853, 584, 24933642]
95437


In [91]:
# Tag 7A
Tree = TreeCrawler()
for batch in PuzzleInputbyCommands:
    Tree.commandOutputParser(batch)
Tree.moveToRoot()
print('Suche beginnt in dieser Node: %s' %Tree.getActiveNode().getName())
sizeOfAllDirectories = Tree.getSizeOfAllSubDirectories()
# print(sizeOfAllDirectories)
Antwort07A = [size for size in sizeOfAllDirectories if size <= 100000]
print(sum(Antwort07A))


# Tag 7B
totalSpace = 70_000_000
neededSpace = 30_000_000
freeSpace = totalSpace-sizeOfAllDirectories[0]
spaceToBeFreed = neededSpace-freeSpace
# print(spaceToBeFreed)
differenceToOptimum = [dir-spaceToBeFreed for dir in sizeOfAllDirectories]
IndexOptimum = differenceToOptimum.index(min([elem for elem in differenceToOptimum if elem >=0]))
# print(IndexOptimum)
AntwortB = sizeOfAllDirectories[IndexOptimum]
print('Antwort B ist: %s' %AntwortB)

root node
Suche beginnt in dieser Node: root
1423358
Antwort B ist: 545729
